In [1]:
import numpy as np
import pandas as pd
import os
import gc

ModuleNotFoundError: No module named 'pandas'

In [2]:
df17 = pd.read_parquet(f's3://sofi-data-science/hpeng/pl-gen4/data_dump/cleaned_tables/uw/all_features_dev1_sample_encoding_v1.parquet')

In [3]:
df18 = pd.read_parquet(f's3://sofi-data-science/hpeng/pl-gen4/data_dump/cleaned_tables/uw/all_features_dev2_sample_encoding_v1.parquet')

In [4]:
df_dev = pd.concat([df17, df18], axis=0, ignore_index=True)

In [8]:
df_dev[df_dev.is_funded == 0]['status_string'].head(100)

15752    None
15753    None
15754    None
15755    None
15756    None
         ... 
15847    None
15848    None
15849    None
15850    None
15851    None
Name: status_string, Length: 100, dtype: object

In [19]:
df_dev['bad1'] = np.where((df_dev['status_string'].str[0:24].str.contains('4|5|6|B|D|W')),1
                                 ,np.where(~df_dev.status_string.isnull(), 0, np.nan))
df_dev['bad_bk'] = np.where((df_dev['status_string'].str[0:24].str.contains('B'))
                          &(df_dev['bad1']!=1),1
                                 ,np.where(~df_dev.status_string.isnull(), 0, np.nan))
df_dev['bad_bk1'] = np.where((df_dev['status_string'].str[0:24].str.contains('B'))
                          ,1
                                 ,np.where(~df_dev.status_string.isnull(), 0, np.nan))
df_dev['bad_bk2'] = np.where((df_dev['status_string'].str[0:24].str.contains('B'))
                          &(df_dev['status_string'].str[0:24].str.contains('4|5|6')),1
                                 ,np.where(~df_dev.status_string.isnull(), 0, np.nan))

In [20]:
df = df_dev[(df_dev.is_funded == 1) & (df_dev.applicant_type=='PRIMARY')].copy()

In [21]:
df[['bad1','bad_bk','bad_bk1','bad_bk2']].mean()

bad1       0.060735
bad_bk     0.000000
bad_bk1    0.007418
bad_bk2    0.001169
dtype: float64

In [18]:
df[df.bad_bk1==1].status_string.head(10)

329          CCCCCCCCCCCBBDWWWWWWWWWWWWWWWWWWWWWWWWWWWWWW
487     CCCCCCCCCCCCCCCC24BDDDWWWWWWWWWWWWWWWWWWWWWWWW...
640     CCCCCCC3BBDDDDWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWW...
835          CCCCCCCCCCCCCCCC2BBDWWWWWWWWWWWWWWWWWWWWWWWW
999             CCCCCCCCCCCCCCCCCCCBBDDWWWWWWWWWWWWWWWWWW
1245         CCCCCCCCCCCCCCCCBBDDDDDDDDDDDDDDDDDDDDDDDDDD
1320      CCCCCCCCCCCCCCCCCCCBBDDDDDDDDDDDDDDDDDDDDDDDDDD
1405    CCCCCCCCCCCCCCC23BDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDD
1453        CCCCCCCCCCCCCCC34BDDDDDDDDDDDDDDDDDDDDDDDDDDD
1482     CCCCCCCCCCCCCCCCCCBBDDDDDDDDDDDDDDDDDDDDDDDDDDDD
Name: status_string, dtype: object

In [22]:
0.001169/0.007418

0.15758964680506873